# Launch Association sequencing on Wynton

In [1]:
from Bio import SeqIO
import config_readwrite as crw
import glob
import os
import sys

In [2]:
config_tag = "config"
config, cfn = crw.read_config(os.path.join(os.path.dirname(os.getcwd()), config_tag))

In [3]:
PATH = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/assoc"
config_tag= "null_mpra.config"
cfn = os.path.join(PATH, config_tag)

if os.path.exists(cfn) is False:
    if os.path.exists(PATH) is False:
         os.system(f"mkdir {PATH}")
    os.system(f"touch {cfn}")
    
config, cfn = crw.read_config(cfn)

# config

## Novogene download

In [4]:
# write novogene info
section = "novogene"
crw.check(config, section)

write_dict = {
        "Host":"sftp://38.122.175.98",
        "Username":"1735-2193",
        "Password":"BRdCV2",
        "Port":"2222",
        }
for key, value in write_dict.items():
    config[section][key]=value

## Parameters

In [5]:
PATH = "/wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA"  # change me

NAME = "null_mpra"  # change me
FASTQ_PREFIX = "SF_asso_S1"  # change me
DESIGN_FILENAME = "15mer.fo.pam.scaffold.ext200.library.TWIST.fa"  #change me

SRC = "/wynton/group/ahituv/fongsl/src/MPRAflow/"
NEXTFLOW_SCRIPT = os.path.join(SRC, "association.nf")
DESIGN = os.path.join(PATH, DESIGN_FILENAME)

MIN_FRAC = "0.7"  # change me
MAPQ = 5  #change me

## add to configuration file

In [7]:
section = "assoc_seq"
crw.check(config, section)

R1= os.path.join(PATH, f"{FASTQ_PREFIX}_R1_001.fastq.gz")
R2= os.path.join(PATH, f"{FASTQ_PREFIX}_R4_001.fastq.gz")
BC= os.path.join(PATH, f"{FASTQ_PREFIX}_R2_001.fastq.gz") # note: SF_asso_S1_R3_001.fastq.gz is common i5 GCCAGCGCCG


# write to config
write_dict = {
    "path":PATH,
    "name":NAME,
    "fastq_prefix":FASTQ_PREFIX, 
    "ins": R1, 
    "inspe": R2, 
    "design_fa": DESIGN, 
    "src": SRC,

    "min_frac": str(MIN_FRAC), 
    "mapq": str(MAPQ), 
    "nextflow_script":NEXTFLOW_SCRIPT,  

}

for key, value in write_dict.items():
    config[section][key]=value
crw.write(config, cfn)

# Command line command - nextflow 

In [8]:
CMD = " ".join([
    "cd", SRC, 
    "|",
    "nextflow run",
    NEXTFLOW_SCRIPT,
    "--w",
    PATH,
    "--name", 
    NAME,
    "--fastq-insert", 
    R1, 
    "--fastq-insertPE", 
    R2, 
    "--design", 
    DESIGN, 
    "--fastq-bc", 
    BC, 
    " --min-frac", 
    MIN_FRAC, 
    "--mapq", 
    str(MAPQ)
    ])
print("REMEMBER TO ACTIVATE YOUR MPRAflow CONDA ENVIRONMENT!\n\n", CMD)

REMEMBER TO ACTIVATE YOUR MPRAflow CONDA ENVIRONMENT!

 cd /wynton/group/ahituv/fongsl/src/MPRAflow/ | nextflow run /wynton/group/ahituv/fongsl/src/MPRAflow/association.nf --w /wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA --name null_mpra --fastq-insert /wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/SF_asso_S1_R1_001.fastq.gz --fastq-insertPE /wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/SF_asso_S1_R4_001.fastq.gz --design /wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/15mer.fo.pam.scaffold.ext200.library.TWIST.fa --fastq-bc /wynton/group/ahituv/fongsl/projects/nullomers/data/20230815_nullomer_MPRA/SF_asso_S1_R2_001.fastq.gz  --min-frac 0.7 --mapq 5


In [ ]:
CMD

    # example from gordon 2020
    
    nextflow run --w <path/to/Basic>/Assoc_Basic/work association.nf 
             --name SRR10800986 
        --fastq-insert "<path/to/Basic>/Assoc_Basic/data/ SRR10800986_1.fastq.gz" 
        --fastq-insertPE "<path/to/Basic>/Assoc_Basic/ data/SRR10800986_3.fastq.gz" 
        --fastq-bc "<path/to/Basic>/Assoc_ Basic/data/SRR10800986_2.fastq.gz" 
        --design "<path/to/Basic>/Assoc_ Basic/data/design.fa"